In [1]:
# Load FaceNet
import tensorflow as tf

model = tf.keras.models.load_model('facenet_keras.h5')
model.summary()
print (model.input)
print (model.output)

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

In [2]:
# Pre Process the data to extract faces and flush to hard drive
import cv2
import os

face_cascade = cv2.CascadeClassifier()
face_cascade.load('haarcascade_frontalface_default.xml')

data_dir = './training/'

for label in os.listdir(data_dir):
    label_dir = data_dir + label + "/"
    for img in os.listdir(label_dir):
        im_load = cv2.imread(label_dir + img, 1)
        faces = face_cascade.detectMultiScale(im_load, scaleFactor=1.05, minNeighbors=5)
        for x, y, w, h in faces:
            if w > 0.2*im_load.shape[1] and h > 0.2*im_load.shape[0]:
                face_res = cv2.resize(im_load[y:y+h,x:x+w,:], (160, 160))
                cv2.imwrite(label_dir + img, face_res)

In [3]:
# get embedding from pre trained model
import numpy as np
def getEmbedding(im_load, model):
    im_load = im_load.astype('float32')
    mean, std = im_load.mean(), im_load.std()
    im_load = (im_load - mean)/std
    im_load = np.expand_dims(im_load, axis=0)
    return model.predict(im_load)

In [4]:
# Prepare training data for SVM classifier
trainX = None
trainY = None
label_num = 0
for label in os.listdir(data_dir):
    label_dir = data_dir + label + "/"
    for img in os.listdir(label_dir):
        im_load = cv2.imread(label_dir + img, 1)
        vector = getEmbedding(im_load, model)
        if trainX is None:
            trainX = np.array(vector)
            trainY = np.array([label_num])
        else:
            trainX = np.append(trainX, vector, axis=0)
            trainY = np.append(trainY, label_num)
    label_num += 1

In [5]:
print(trainX.shape)
print(trainY.shape)

(12, 128)
(12,)


In [6]:
# using SVM classifier
from sklearn.svm import SVC

classifier = SVC(kernel='linear', probability=True)
classifier.fit(trainX, trainY)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [7]:
classifier.predict(trainX)

array([0, 0, 0, 0, 1, 1, 2, 2, 3, 3, 3, 3])

In [ ]:
# Face Recognition in live video stream
video = cv2.VideoCapture(0)
frames = 0
label, label_map = 0, ['Arjun', 'Shaveta', 'Bunty', 'Yuvraj']

while True:
    check, frame = video.read()
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.05, minNeighbors=5, minSize=(150, 150))
    for x, y, w, h in faces:
        if frames % 20 == 0:
            face_res = cv2.resize(frame[y:y+h,x:x+w,:], (160, 160))
            vector = getEmbedding(face_res, model)
            label = classifier.predict(vector)[0]
        cv2.putText(frame, label_map[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
        frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
    
    cv2.imshow("Video", frame)
    frames += 1
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
        
video.release()
cv2.destroyAllWindows()